# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 8: Decision Trees (2)


We will implement a trainer for the decision tree. If you completed Assignment 7, then you should have most of the code already. Now we will put it in the right place.

The trainer implementation will happen in the following stages:

1. Create a class ```NumericSplitter``` which finds the best threshold for a numeric feature and outputs a ```NumericSplit```
2. Create a class ```DecisionTree``` which implements a recursive trainer given a list of possible splits
3. Add a ```predict``` method to the decision tree class that predicts for new examples

After building a decision tree, you will implement a few methods that we will use in future assignments to evaluate trained models.


---
** Setup **

In [154]:
require 'test/unit/assertions'
require 'daru'
require 'distribution'
require 'json'

include Test::Unit::Assertions

## Loads data files
def read_sparse_data_from_csv prefix
  data = []
  classes = Hash.new {|h,k| h[k] = 0}
  header = File.read(prefix + ".header").chomp.split(",")  
  
  File.open(prefix + ".csv").each_line.with_index do |l, i|
    a = l.chomp.split ","
    next if a.empty?
    row = {"features" => Hash.new}
    
    header.each.with_index do |k, i|
      v = a[i].to_f
      if k == "label"
        row["label"] = v.to_i
      else
        next if v.zero?
        row["features"][k] = v
      end
    end
    classes[row["label"]] += 1
    
    data << row
  end
  return {"classes" => classes, "features" => header[0,header.size - 1], "data" => data}
end

iris = read_sparse_data_from_csv "iris"
spambase = read_sparse_data_from_csv "spambase"
spambase.size

3

## Question 1 (20 Points)

Copy **your** solutions from [assignment-7](../assignment-7/assignment-7.ipynb) here.

---

In [155]:
def class_distribution dataset
   # BEGIN YOUR CODE
  class_group = dataset.group_by{|row| row["label"]}
  class_dist = Hash.new
  ## size of rows
  total_size = dataset.size
  class_num = class_group.size
  
  class_group.each_key do |key|
    class_dist[key] = (class_group[key].size.to_f / total_size.to_f).to_f
  end
  class_dist
  #END YOUR CODE
end

:class_distribution

In [156]:
### TESTS ###
# Check that each class has a probability 1/3
t1_iris_dist = class_distribution iris["data"]
t1_iris_num_classes = 3
t1_iris_num_classes.times do |cls|
  assert_in_delta t1_iris_dist[cls], 0.33333, 1e-4
end

3

In [177]:
def entropy dist
  # BEGIN YOUR CODE
  sum = dist.values.reduce(0.0, :+)
  entropy = 0.0
  dist.values.each do |d|
    if d != 0
      entropy -= (d / sum) * Math.log(d / sum)
    end
  end
  entropy
#END YOUR CODE
end

:entropy

In [178]:
### TESTS ###
#Checks the class entropy for Spambase dataset
t2_spambase_dist = class_distribution spambase["data"]
t2_spambase_entropy = entropy t2_spambase_dist
assert_equal 0.6705230209876485, t2_spambase_entropy, 1e-4

## Question 2.1 (10 points)

Implement the ```NumericSplit``` split class which splits a dataset given a feature (```@fname```) and threshold (```@value```) into two datasets. The class template below provides a string representation for the split that will be used later. Note that code required was part of Assignment 2.


In [179]:
class NumericSplit
  attr_reader :fname, :value
  def initialize fname, value
    @fname = fname
    @value = value
    @lpath = "#{@fname} < #{@value}"
    @rpath = "#{@fname} >= #{@value}"
  end
  
  def to_s
    "Numeric[#{@fname} <=> #{@value}]"
  end
  
  def split dataset
    # BEGIN YOUR CODE
    res = Hash.new
    res[@lpath] = Array.new
    res[@rpath] = Array.new
    
    dataset.each do |row|
      if row["features"][@fname] != nil
        if row["features"][@fname] < @value
          res[@lpath] << row
        else
          res[@rpath] << row
        end
      else
        res[@lpath] << row
      end
    end
    res
    #END YOUR CODE
  end
end

:split

In [180]:
### TEST ###

t21_split = NumericSplit.new "petal_length", 1.7

# Checks for the number of examples when split on petal_length = 1.7
t21_iris_splits = t21_split.split iris["data"]
t21_split_sizes = t21_iris_splits.values.collect {|v| v.size}.sort
t21_num_splits = 2
assert_equal t21_num_splits, t21_split_sizes.size
assert_equal 44, t21_split_sizes[0]
assert_equal 106, t21_split_sizes[1]

## Question 2.2 (5 points)

Add a ```test``` function to the ```NumericSplit``` which returns the name of the left path or right path depending on the value. This will be used in the prediction stage. Note that we are "re-opening" a class to add a new method, which is a feature of Ruby.

In [181]:
class NumericSplit
  def test x
  # BEGIN YOUR CODE
    return ((x["features"][@fname] == nil) or (x["features"][@fname] < @value)) ? @lpath : @rpath
  #END YOUR CODE
  end

end

:test

In [182]:
### TEST ###

t22_split = NumericSplit.new "petal_length", 1.7
t22_x_left = {"features" => {"petal_length" => 0.7}}
assert_equal "petal_length < 1.7", t22_split.test(t22_x_left)

t22_x_right = {"features" => {"petal_length" => 1.71}}
assert_equal "petal_length >= 1.7", t22_split.test(t22_x_right)

t22_x_right_eq = {"features" => {"petal_length" => 1.7}}
assert_equal "petal_length >= 1.7", t22_split.test(t22_x_right_eq)

# Question 2.3 (10 points)

Implement the ```NumericSplitter``` which finds the threshold with best information gain for a numeric feature and returns a ```NumericSplit```. Remember that, sometimes, a feature may not be numeric. Each splitter has a ```matches?(x)``` function to check this. 



In [183]:
class NumericSplitter
  def new_split dataset, initial_entropy, fname
    # BEGIN YOUR CODE
    t_max = 0.0
    ig_max = 0.0
  
    split_l = Hash.new(0)
    split_r = Hash.new(0)
  
    sorted_x = dataset.sort_by{|row| row["features"].has_key?(fname) ? row["features"][fname] : 0}
  
    sorted_x.each do |row|
      split_r[row["label"]] += 1
    end
    size = sorted_x.size
    ig_max = 0
    sorted_x.each_with_index do |row, index|
      split_l[row["label"]] += 1
      split_r[row["label"]] -= 1
      if(index + 1 < size and row["features"][fname] == sorted_x[index + 1]["features"][fname])
        next
      end 
      p1 = (index + 1.0)/ size
      p2 = (size - index - 1.0)/ size
      ig = initial_entropy - p1 * entropy(split_l) - p2 * entropy(split_r)
      if ig > ig_max
        ig_max = ig
        t_max = sorted_x[index + 1]["features"][fname]
      end
    end
    #END YOUR CODE
    [NumericSplit.new(fname, t_max), ig_max]
  end
  
  def matches? x, fname
    x.all? {|r| r["features"].fetch(fname, 0.0).is_a?(Numeric)}
  end
end


:matches?

In [184]:
### TEST ###
t23_iris_entropy = entropy(class_distribution(iris["data"]))
t23_splitter = NumericSplitter.new 
t23_split_result = t23_splitter.new_split iris["data"], t23_iris_entropy, "sepal_width"

t23_split, t23_info_gain = t23_split_result
assert_not_nil t23_split
assert_in_delta 0.18570201019349364, t23_info_gain, 1e-2

In [185]:
### TEST ### 
# Checks split value
assert_in_delta 3.4, t23_split.value, 1e-2

## Question 3.1 (10 Points)

After initializing your decision tree, a core method is to find the best split. Going through each feature, find the best split in each feature and the best of those feature. Return the best (split, information gain) pair or a pair of (nil, 0) to indicate that there is no best split.

This cell contains the class template that will be used later. Note that the tree has "symbols" and not "strings" for the names of the fields.

In [186]:
class DecisionTree
  attr_reader :tree, :h0
  
  def initialize splitters, min_size, max_depth
    @splitters = splitters
    @min_size = min_size
    @max_depth = max_depth
  end
  
  def init_dataset dataset
    @dataset = dataset
    @header = @dataset["features"]
    @c_dist = class_distribution @dataset["data"]
    @h0 = entropy @c_dist
    @tree = {n: @dataset["data"].size, entropy: @h0, dist: @c_dist, split: nil, children: {}}    
  end
  
  def find_best_split dataset, initial_entropy
    # BEGIN YOUR CODE
    ig_best = 0.0
    split_obj_best = nil
    
    fnames = Set.new
    dataset.each do |row|
      fnames = fnames | row["features"].keys.to_set
    end
    
    @splitters.each do |splitter|
      fnames.each do |fname|
        if splitter.matches? dataset, fname 
          num_split_obj, ig = splitter.new_split dataset, initial_entropy, fname
          if ig > ig_best
            split_obj_best = num_split_obj
            ig_best = ig
          end
        end
      end
    end
    return [split_obj_best, ig_best]
    #END YOUR CODE
  end
end

:find_best_split

In [187]:
### TEST ###

# Check the first split for iris
t31_iris_entropy = entropy(class_distribution(iris["data"]))
t31_decision_tree = DecisionTree.new [NumericSplitter.new], 10, 2
t31_decision_tree.init_dataset iris

t31_split, t31_info_gain = t31_decision_tree.find_best_split iris["data"], t31_decision_tree.h0

assert_not_nil t23_split
assert_in_delta 0.6365141682948128, t31_info_gain, 1e-2


In [188]:
### TEST ###
# Check the name and value for the first split for iris
assert_equal "petal_length", t31_split.fname
assert_equal 3.0, t31_split.value

## Question 3.2 (10 Points)

Now we can build the decision tree. First we will initialize the class with a set of spliters and parameters for the smallest size of a leaf and the max_depth.

The tree structure consists of the root node, which has the following structure:
```
@tree = {n: @dataset["data"].size, entropy: @h0, dist: @c_dist, split: nil, children: {}}
```

The next level of nodes are contain in the ```children``` of the root node. Each child has a key for the corresponding path for the ```split```, which you implemented in ```NumericSplit``` above. 

The ```build_tree``` is a recursive algorithm which checks for two termination conditions:
1. If we have reached ```max_depth```, then return immmedately. A root-only tree as a depth of 1.
1. If the number of examples in the node is too small

Otherwise, recursively build the tree on the children of the root.

In [189]:
class DecisionTree
  def train dataset
    init_dataset dataset
    build_tree @dataset["data"], @tree, @max_depth
  end

  def build_tree x, root, max_depth
    # BEGIN YOUR CODE
    if x == nil or max_depth == 1 or root[:n] < @min_size
      return
    end
    
    split_best, ig_best = find_best_split x, root[:entropy]
    if split_best == nil
      return 
    end
    
    split_x = split_best.split x 
    root[:split] = split_best
    
    keys = split_x.keys
    l_key = keys[0]
    l_data = split_x[l_key]
    l_size = l_data.size
    l_dist = class_distribution l_data
    l_entropy = entropy l_dist
    left_tree = {n: l_size, entropy: l_entropy, dist: l_dist, split: nil, children: {}}
    
    r_key = keys[1]
    r_data = split_x[r_key]
    r_size = r_data.size
    r_dist = class_distribution r_data
    r_entropy = entropy r_dist
    right_tree = {n: r_size, entropy: r_entropy, dist: r_dist, split: nil, children: {}}
    
    
    root[:children] = {l_key =>left_tree, r_key => right_tree}
    
    build_tree l_data, left_tree, max_depth - 1
    build_tree r_data, right_tree, max_depth - 1
    
    #END YOUR CODE
  end
end

:build_tree

In [190]:
### TEST ###

# Builds a root-only tree and validates that there are no children
t32_model = DecisionTree.new [NumericSplitter.new], 10, 1
t32_model.train iris
t32_tree = t32_model.tree

puts "Root Only Tree", JSON.pretty_generate(t32_tree)

assert_nil t32_tree[:split]
assert_true t32_tree[:children].empty?

Root Only Tree
{
  "n": 150,
  "entropy": 1.0986122886681096,
  "dist": {
    "1": 0.3333333333333333,
    "2": 0.3333333333333333,
    "0": 0.3333333333333333
  },
  "split": null,
  "children": {
  }
}


In [191]:
### TEST ###

# Builds a tree with one split and checks the split
t32_model_2 = DecisionTree.new [NumericSplitter.new], 10, 2
t32_model_2.train iris
t32_tree_2 = t32_model_2.tree

puts "Two-level Tree", JSON.pretty_generate(t32_tree_2)

assert_not_nil t32_tree_2[:split]
assert_equal "petal_length", t32_tree_2[:split].fname
assert_equal 3.0, t32_tree_2[:split].value
assert_equal 2, t32_tree_2[:children].size
assert_true(t32_tree_2[:children].values.all? {|c| c[:split].nil?})

Two-level Tree
{
  "n": 150,
  "entropy": 1.0986122886681096,
  "dist": {
    "1": 0.3333333333333333,
    "2": 0.3333333333333333,
    "0": 0.3333333333333333
  },
  "split": "Numeric[petal_length <=> 3.0]",
  "children": {
    "petal_length < 3.0": {
      "n": 50,
      "entropy": 0.0,
      "dist": {
        "0": 1.0
      },
      "split": null,
      "children": {
      }
    },
    "petal_length >= 3.0": {
      "n": 100,
      "entropy": 0.6931471805599453,
      "dist": {
        "1": 0.5,
        "2": 0.5
      },
      "split": null,
      "children": {
      }
    }
  }
}


In [192]:
### TEST ###

# Builds a full tree with a large depth and checks a few paths
t32_model_3 = DecisionTree.new [NumericSplitter.new], 25, 10
t32_model_3.train iris
t32_tree_3 = t32_model_3.tree

puts "Multi-level Tree", JSON.pretty_generate(t32_tree_3)

assert_equal "petal_length", t32_tree_3[:split].fname
assert_equal "petal_width", t32_tree_3[:children]["petal_length >= 3.0"][:split].fname
assert_equal "petal_length", t32_tree_3[:children]["petal_length >= 3.0"][:children]["petal_width < 1.8"][:split].fname


Multi-level Tree
{
  "n": 150,
  "entropy": 1.0986122886681096,
  "dist": {
    "1": 0.3333333333333333,
    "2": 0.3333333333333333,
    "0": 0.3333333333333333
  },
  "split": "Numeric[petal_length <=> 3.0]",
  "children": {
    "petal_length < 3.0": {
      "n": 50,
      "entropy": 0.0,
      "dist": {
        "0": 1.0
      },
      "split": null,
      "children": {
      }
    },
    "petal_length >= 3.0": {
      "n": 100,
      "entropy": 0.6931471805599453,
      "dist": {
        "1": 0.5,
        "2": 0.5
      },
      "split": "Numeric[petal_width <=> 1.8]",
      "children": {
        "petal_width < 1.8": {
          "n": 54,
          "entropy": 0.30849545083110386,
          "dist": {
            "1": 0.9074074074074074,
            "2": 0.09259259259259259
          },
          "split": "Numeric[petal_length <=> 5.0]",
          "children": {
            "petal_length < 5.0": {
              "n": 48,
              "entropy": 0.10126481756679193,
              "dist":

## Question 3.3 (10 Points)

Now, add a recursive evaluator for an example. This should return the class name with the highest posterior probability within the leaf. In the event of a tie, take the smallest class index. The score is not important for the following

In [193]:
class DecisionTree
  def predict x
    return eval_tree x, @tree
  end
  
  def eval_tree x, root
    # BEGIN YOUR CODE
    if root[:children].empty?
      return root[:dist].key(root[:dist].values.max)
    end
    path = root[:split].test x
    child = root[:children][path]
    return eval_tree x, child
    #END YOUR CODE
  end
end



:eval_tree

In [194]:
### TEST ###

# Tests the evaluator with the two-level tree
t33_model = DecisionTree.new [NumericSplitter.new], 10, 2
t33_model.train iris
t33_tree = t33_model.tree

t33_x1 = {"features" => {"petal_length" => 1.2}}
assert_equal 0, t33_model.predict(t33_x1)

t33_x2 = {"features" => {"petal_length" => 3.1}}
assert_equal 1, t33_model.predict(t33_x2)


## Question 4.1 (10 Points)

Create a "confusion matrix" for your evaluation dataset in which the rows are the predicted label and columns are the actual label. The value contains the number of examples predicted in the row class but actually in the column class. The input to this function is dataset, containing the original labels, and an array of predictions.

In [195]:
def confusion_matrix dataset, predictions
  # BEGIN YOUR CODE
  classes = dataset["classes"]
  class_size = classes.size
  conf_matrix =Array.new(class_size) { Array.new(class_size,0) }
  
  data = dataset["data"]
  data.each_with_index do |row, index|
    predictions.each_with_index do |predict, p_index| 
      if index == p_index
        conf_matrix[predict][row["label"]] += 1
      end
    end
  end
  conf_matrix
  
  # END YOUR CODE
end


:confusion_matrix

In [196]:
### TEST ###

# Create a test dataset with no features (no required for evaluation) and two labels
t41_dataset = {
  "classes" => [0,1],
  "data" => [
    {"features" => {}, "label" => 0},
    {"features" => {}, "label" => 0},
    {"features" => {}, "label" => 1}
  ]
}
#Assume the classifier predicted these labels
t41_predictions = [0,1,1]

t41_mat = confusion_matrix t41_dataset, t41_predictions
assert_equal 2, t41_mat.size
assert_equal 1, t41_mat[0][0]
assert_equal 0, t41_mat[0][1]
assert_equal 1, t41_mat[1][1]

## Question 4.2 (10 Points)

Let's evaluate your models. Compute the accuracy, which is the proportion of correct answers.

In [197]:
def accuracy conf_mat
  # BEGIN YOUR CODE
  total_size = 0
  true_size = 0
  conf_mat.each_with_index do |row, i|
    row.each_with_index do |val, j|
      if i == j
        true_size += val
      end
      total_size += val
    end
  end
  acc = true_size.to_f / total_size 
  acc
  #END YOUR CODE
end

:accuracy

In [198]:
### TEST ### 

# Tests accuracy for the example above
t41_dataset = {
  "classes" => [0,1],
  "data" => [
    {"features" => {}, "label" => 0},
    {"features" => {}, "label" => 0},
    {"features" => {}, "label" => 1},
    {"features" => {}, "label" => 1}  
  ]
}
t41_predictions = [0,1,1,1]
t41_mat = confusion_matrix t41_dataset, t41_predictions
assert_equal 0.75, accuracy(t41_mat)

## Question 4.3 (5 Points)

Calculate the cross-validation accuracy of the decision tree on the iris dataset. In $k$-fold cross-validation, the dataset is divided into $k$ equal-sized partitions. In each of $k$ folds, each $k-1$ of the partitions is used as a training set and the remaining partition is the test set. The model is trained on the training set and evaluated on the testing set. The average and standard deviation of accuracy is reported.

Cross validation has been implemented for you. All you need to do is run the training and testing parts and fill in the accuracy array.

In [203]:
def cross_validate data, folds, &block
  dataset = data["data"]
  fold_size = dataset.size / folds
  subsets = []
  dataset.shuffle.each_slice(fold_size) do |subset|
    subsets << subset
  end
  i_folds = Array.new(folds) {|i| i}
  
  i_folds.collect do |fold|
    test = subsets[fold]
    train = (i_folds - [fold]).flat_map {|t_fold| subsets[t_fold]}
    train_data = data.clone
    train_data["data"] = train
    
    test_data = data.clone
    test_data["data"] = test
    
    yield train_data, test_data, fold
  end
end

def mean x
  sum = x.inject(0.0) {|u,v| u += v}
  sum / x.size
end

def stdev x
  m = mean x
  sum = x.inject(0.0) {|u,v| u += (v - m) ** 2.0}
  Math.sqrt(sum / (x.size - 1))
end

:stdev

In [210]:
def cross_validation_accuracy iris, folds = 10, min_size = 10, max_depth = 50
  acc_arr = Array.new
  cross_validate iris, folds do |train, test|
    # BEGIN YOUR CODE
    dec_tree = DecisionTree.new [NumericSplitter.new], min_size, max_depth
    dec_tree.train train
    pred_arr = Array.new
    data = test["data"]
    data.each do |row|
      pred_arr << dec_tree.predict(row)
    end
    mat = confusion_matrix test, pred_arr
    acc_arr << accuracy(mat)
    #END YOUR CODE
  end
  acc_arr
end


:cross_validation_accuracy

In [211]:
### TEST ###

# Iris accuracy should be 93% +/- 6%
t43_iris_accuracy = cross_validation_accuracy iris
assert_equal 10, t43_iris_accuracy.size
assert_in_delta 0.93, mean(t43_iris_accuracy), 0.1
assert_in_delta 0.06, stdev(t43_iris_accuracy), 0.1

In [212]:
### TEST ###

# Spambase accuracy should be 85% +/- 1%
# Note that this will take a few minutes
t43_spambase_accuracy = cross_validation_accuracy spambase, 5
assert_equal 5, t43_spambase_accuracy.size
assert_in_delta 0.85, mean(t43_spambase_accuracy), 0.1
assert_in_delta 0.01, stdev(t43_spambase_accuracy), 0.05

## Question 4.4 (2 points)

Varying the value $\eta$, the proportion of the training set to use as the ```min_size``` parameter, determine which value of $\eta$ has the highest mean in 10-fold cross validation on the **iris** dataset. 

In [213]:
etas = [0.05, 0.10, 0.15, 0.20, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]
accuracy_for_eta = etas.collect do |eta|
  # BEGIN YOUR CODE
  
  #END YOUR CODE
end

[nil, nil, nil, nil, nil, nil, nil, nil, nil, nil, nil, nil, nil, nil, nil, nil, nil, nil]

In [214]:
### TEST ### 

assert_equal 18, accuracy_for_eta.size
assert_true accuracy_for_eta.all? {|a| 0 < a && a < 1}
assert_true accuracy_for_eta.min < 0.6
assert_true accuracy_for_eta.max > 0.9

Daru::DataFrame.new({x: etas, y: accuracy_for_eta}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Eta"
  plot.y_label "Iris Accuracy"
end

ArgumentError: comparison of Integer with nil failed

## Question 4.5 (3 Points)

Answer the following questions in the function below by setting your answers in an array. Example
```
questions = ["What is 2 + 2? Is it 4 ("A") or 22 ("B")"]
answers = ["A"]
```

If you change the question, you will not get credit for the answer.

In [215]:
def q45_answer(qs)
  questions = [
    "Does accuracy decrease ('A') or increase ('B') as we increase eta?",
    "Do you think that this trend is particular to this dataset ('A') or common among other datasets ('B')",
    "What property does the eta control? Overfitting ('A') or Convergence ('B')",    
  ]

  # BEGIN YOUR CODE
  return ["B","B","B"]
  #END YOUR CODE
  answers.each.with_index {|answer, question_id| qs[questions[question_id]] = answer}
end

:q45_answer

In [216]:
### HIDDEN TEST ###
